In [ ]:
#https://colab.research.google.com/drive/1yFphU6PW9Uo6lmDly_ud9a6c4RCYlwdX#scrollTo=j4DKDtFwiF0S
#http://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
#https://github.com/arushiprakash/MachineLearning/blob/main/BERT%20Word%20Embeddings.ipynb

In [1]:
!pip install transformers==3
!pip install sentence_transformers

     |████████████████████████████████| 757kB 11.0MB/s 
     |████████████████████████████████| 3.0MB 24.4MB/s 
     |████████████████████████████████| 890kB 40.8MB/s 
     |████████████████████████████████| 1.2MB 34.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5650e0d950767934db7dbfdce5415c17f018d1dcd0d56e431a47cf2fde048d9a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 71kB 5.6MB/s 
     |████████████████████████████████| 1.8MB 16.6MB/s 
     |████████████████████████████████| 3.2MB 32.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=6d3d4aa6c73156b3d23c0163abb4417bdada010f7eb50b6c4c726f9f093a459e
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
Successfully built se

In [2]:
#http://mccormickml.com/2019/07/22/BERT-fine-tuning/
#https://github.com/abhimishra91/pytorch-tutorials/blob/master/lecture_08_a.ipynb
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [21]:
# Importing stock ml libraries
import numpy as np
import pandas as pd


import transformers
import torch
from transformers import BertTokenizer, BertModel, BertConfig
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:14<00:00, 28.0MB/s]


In [7]:
raw_data_path = '/content/drive/My Drive/NLP/ResearchArticlesClassification/Dataset/raw_train.csv'
#destination_folder = '/content/drive/My Drive/NLP/ResearchArticlesClassification/Dataset'

df_raw =  pd.read_csv(raw_data_path, encoding='utf8')

df_raw.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [8]:
df_raw.shape

(20972, 9)

In [9]:
document_embeddings = sbert_model.encode(df_raw['TITLE'])

In [10]:
document_embeddings.shape

(20972, 768)

In [11]:
document_embeddings

array([[-0.51906806, -0.5829191 ,  1.8336611 , ..., -0.5023897 ,
        -0.5179823 ,  0.50584596],
       [ 0.28318286, -0.94039214,  1.7706263 , ..., -0.7222829 ,
        -1.2445499 ,  0.18752652],
       [-0.44547525, -0.5395652 ,  1.3178205 , ...,  0.24702398,
        -0.7015718 ,  0.50052345],
       ...,
       [-0.01401251,  0.4710028 ,  0.7146599 , ...,  0.22713855,
         0.19967255,  0.8573519 ],
       [-0.8528394 , -0.37673187,  1.1304449 , ...,  0.01878861,
        -0.5628012 ,  0.39648065],
       [-0.30087748,  0.24345386,  1.118451  , ..., -0.4554226 ,
        -1.1595879 ,  0.02022632]], dtype=float32)

In [34]:
with open('/content/drive/My Drive/NLP/ResearchArticlesClassification/embeddings_title_search.npy', 'wb') as f:
    np.save(f, document_embeddings)

**Search Projects **

In [15]:
def search_project_title(query_term, all_title_embeddings, topn):
 # query_embedding = np.array((768))
  query_embedding = sbert_model.encode(query)
  #print(type(query_embeddings))
  cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]
  top_results = torch.topk(cos_scores, k= topn)

  print("Query:", query)
  print("\nTop {} most similar sentences in corpus:".format(topn))

  for score, idx in zip(top_results[0], top_results[1]):
    print(df_raw.iloc[idx.item()]['TITLE'], ":"  , "(Score: {:.4f})".format(score.item()))

In [26]:
query = 'Valid Inference Corrected for Outlier Removal'
search_project_title(query, document_embeddings, 100)

Query: Valid Inference Corrected for Outlier Removal

Top 100 most similar sentences in corpus:
Valid Inference Corrected for Outlier Removal : (Score: 1.0000)
Efficient boundary corrected Strang splitting : (Score: 0.8487)
Efficient Estimation for Dimension Reduction with Censored Data : (Score: 0.8367)
Optimal stopping via reinforced regression : (Score: 0.8279)
Outlier Detection by Consistent Data Selection Method : (Score: 0.8267)
Efficient Structured Surrogate Loss and Regularization in Structured Prediction : (Score: 0.8255)
Faithful Inversion of Generative Models for Effective Amortized Inference : (Score: 0.8121)
Nearly Optimal Adaptive Procedure with Change Detection for Piecewise-Stationary Bandit : (Score: 0.8100)
Rate-optimal Meta Learning of Classification Error : (Score: 0.8073)
Fault Tolerant Consensus Agreement Algorithm : (Score: 0.8017)
Reliable Decision Support using Counterfactual Models : (Score: 0.8012)
Positive semi-definite embedding for dimensionality reduction

In [27]:
query = 'deep learning'
search_project_title(query, document_embeddings, 50)

Query: deep learning

Top 50 most similar sentences in corpus:
Deep Reinforcement Learning that Matters : (Score: 0.8808)
On the Origin of Deep Learning : (Score: 0.8684)
Deep Learning the Ising Model Near Criticality : (Score: 0.8630)
Variational Continual Learning : (Score: 0.8519)
Deep Active Learning for Named Entity Recognition : (Score: 0.8325)
Subregular Complexity and Deep Learning : (Score: 0.8323)
Interpretable Active Learning : (Score: 0.8318)
Deep Generative Learning via Variational Gradient Flow : (Score: 0.8292)
Resolution and Relevance Trade-offs in Deep Learning : (Score: 0.8287)
On Generalization and Regularization in Deep Learning : (Score: 0.8285)
Deep Generative Models with Learnable Knowledge Constraints : (Score: 0.8179)
Learning Deep Representations with Probabilistic Knowledge Transfer : (Score: 0.8157)
Deep Hyperspherical Learning : (Score: 0.8102)
Deep Asymmetric Multi-task Feature Learning : (Score: 0.8084)
Deep Multi-view Learning to Rank : (Score: 0.8070)
L

**Project Title Recommendation**

In [22]:
pairwise_similarities = cosine_similarity(document_embeddings)

In [38]:
len(pairwise_similarities)

20972

In [39]:
pairwise_similarities

array([[1.0000001 , 0.63590693, 0.5457958 , ..., 0.39354774, 0.4224286 ,
        0.51793313],
       [0.63590693, 1.0000001 , 0.6131197 , ..., 0.23797047, 0.43363768,
        0.47535995],
       [0.5457958 , 0.6131197 , 1.        , ..., 0.19788536, 0.58194935,
        0.45909613],
       ...,
       [0.39354774, 0.23797047, 0.19788536, ..., 1.0000001 , 0.30319762,
        0.22987626],
       [0.4224286 , 0.43363768, 0.58194935, ..., 0.30319762, 1.        ,
        0.40768015],
       [0.51793313, 0.47535995, 0.45909613, ..., 0.22987626, 0.40768015,
        0.9999998 ]], dtype=float32)

In [43]:
with open('/content/drive/My Drive/NLP/ResearchArticlesClassification/embeddings_recommendation.npy', 'wb') as f:
    np.save(f, pairwise_similarities)

In [ ]:
#a = pairwise_similarities[0][pairwise_similarities[0] > 0.65]

In [41]:
def most_similar(doc_id, similarity_matrix, matrix, topn, confidence_level=0.75):
    title_name = df_raw.iloc[doc_id]["TITLE"]
    print (f'Document: {title_name}')
    print ('\n')
    print (f'Similar Projects for {title_name}:')
    if matrix=='Cosine Similarity':
        #a = pairwise_similarities[doc_id][pairwise_similarities[doc_id] > confidence_level]
        #print(a)
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
      
    for ix in similar_ix[:topn + 1]:
        if ix==doc_id:
            continue
        #print (similarity_matrix[doc_id][ix])
        a = similarity_matrix[doc_id][ix]
        if a > confidence_level:
          print (f'Document: {df_raw.iloc[ix]["TITLE"]}')
          print (f'{matrix} : {similarity_matrix[doc_id][ix]}\n')
          

In [42]:
most_similar(120,pairwise_similarities,'Cosine Similarity', 10, 0.75)

Document: Fault Tolerant Consensus Agreement Algorithm


Similar Projects for Fault Tolerant Consensus Agreement Algorithm:
Document: Rate-optimal Meta Learning of Classification Error
Cosine Similarity : 0.8287529945373535

Document: Outlier Detection by Consistent Data Selection Method
Cosine Similarity : 0.8206487894058228

Document: Guaranteed Fault Detection and Isolation for Switched Affine Models
Cosine Similarity : 0.813323438167572

Document: Analysis of Distributed ADMM Algorithm for Consensus Optimization in Presence of Error
Cosine Similarity : 0.8038772940635681

Document: Valid Inference Corrected for Outlier Removal
Cosine Similarity : 0.8017245531082153

Document: Discrete Choice and Rational Inattention: a General Equivalence Result
Cosine Similarity : 0.7992140650749207

Document: Efficient Estimation for Dimension Reduction with Censored Data
Cosine Similarity : 0.7958939671516418

Document: Efficient Structured Surrogate Loss and Regularization in Structured Predict